In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
from functools import partial
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.ensemble import RandomSurvivalForest
from survlime.survlime_explainer import SurvLimeExplainer
from survlime.load_datasets import RandomSurvivalData
from xgbse import XGBSEKaplanNeighbors
import pandas as pd


# Generate data

In [ ]:
# Generate data
n_points = 500
true_coef = [1, 1]
r = 1
center = [0, 0]
prob_event = 0.9
lambda_weibull = 10**(-6)
v_weibull = 2
n_features = len(true_coef)

rsd = RandomSurvivalData(
    center=center,
    radius=r,
    coefficients=true_coef,
    prob_event=prob_event,
    lambda_weibull=lambda_weibull,
    v_weibull=v_weibull,
    time_cap=None,
    random_seed=90,
)

# Train
X, time_to_event, delta = rsd.random_survival_data(num_points=n_points)
z = [(d, t) for d, t in zip(delta, time_to_event)]
y = np.array(z, dtype=[("delta", np.bool_), ("time_to_event", np.float32)])
total_row_train = X.shape[0]
print('total_row_train:', total_row_train)


In [ ]:
# Point to explain
x_new = np.array([0.1, 0.1])
x_new_array = np.expand_dims(x_new, axis=0)

# Cox

In [ ]:
# Fit a Cox model
model = CoxPHSurvivalAnalysis()
model.fit(X, y)
print(model.coef_)

In [ ]:
output_array = model.predict_cumulative_hazard_function(x_new_array, return_array=True)
output_array_2 = model.predict_cumulative_hazard_function(x_new_array)
print(output_array_2[0](output_array_2[0].x[:5]))
print(output_array[0][:5])

In [ ]:
# SurvLime for COX
explainer = SurvLimeExplainer(
    training_data=X,
    target_data=y,
    model_output_times=model.event_times_,
    sample_around_instance=True,
    random_state=10,
)

b, result = explainer.explain_instance(
    data_row=x_new,
    predict_fn=partial(model.predict_cumulative_hazard_function, return_array=True),
    num_samples=1000,
)

print('b:', b)

In [ ]:
# SurvLime for COX
explainer = SurvLimeExplainer(
    training_data=X,
    target_data=y,
    model_output_times=model.event_times_,
    sample_around_instance=True,
    random_state=10,
)

b, result = explainer.explain_instance(
    data_row=x_new,
    predict_fn=model.predict_cumulative_hazard_function,
    num_samples=1000,
)

print('b:', b)

# Random forest

In [ ]:
model = RandomSurvivalForest().fit(X, y)

In [ ]:
# SurvLime for RSF
explainer = SurvLimeExplainer(
    training_data=X,
    target_data=y,
    model_output_times=model.event_times_,
    sample_around_instance=True,
    random_state=10,
)

b, result = explainer.explain_instance(
    data_row=x_new,
    predict_fn=partial(model.predict_cumulative_hazard_function, return_array=True),
    num_samples=1000,
    verbose=True,
)

print('b:', b)

In [ ]:
explainer = SurvLimeExplainer(
    training_data=X,
    target_data=y,
    model_output_times=model.event_times_,
    sample_around_instance=True,
    random_state=10,
)

b, result = explainer.explain_instance(
    data_row=x_new,
    predict_fn=model.predict_cumulative_hazard_function,
    num_samples=1000,
)

print('b:', b)

# xgboostse
https://loft-br.github.io/xgboost-survival-embeddings/how_xgbse_works.html

In [ ]:
X_df = pd.DataFrame(X, columns = ['A', 'B'])

In [ ]:
xgbse_model = XGBSEKaplanNeighbors(n_neighbors=50)
xgbse_model.fit(X_df, y)

In [ ]:
x_new_array_df = pd.DataFrame(x_new_array, columns = ['A', 'B'])

In [ ]:
explainer = SurvLimeExplainer(
    training_data=X_df,
    target_data=y,
    model_output_times=xgbse_model.time_bins,
    sample_around_instance=True,
    random_state=10,
)

b, result = explainer.explain_instance(
    data_row=x_new_array_df,
    predict_fn=xgbse_model.predict,
    num_samples=1000,
)

print('b:', b)